In [1]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
#read the input
carDf= pd.read_csv('newCardata.csv')
carDate= pd.read_csv('dataWithDate.csv')
carLable= carDate['FraudFound']

#drop the Accident Date and Claim Date
carDate.drop(['FraudFound'],inplace=True,axis=1)
carDate.drop(['AccidentDate'],inplace=True,axis=1)
carDate.drop(['ClaimDate'],inplace=True,axis=1)

carDate.head()

print(type(carLable[0]))

<class 'numpy.int64'>


In [3]:

#select all the attributes of type object
car_object= carDate.select_dtypes(include=['object']).copy()

#drop the attributes of type object
carDate.drop(carDate.select_dtypes(['object']),inplace=True,axis=1)

carDate.head()
#print(type(carDate))

,RepNumber,Deductible,DriverRating,DayDiff
0,12,300,1,0
1,15,400,4,5
2,7,400,3,13
3,4,400,2,20
4,3,400,1,15


In [4]:
#one-hot representation preprocessing

car_copy = car_object.copy()
car_copy = pd.get_dummies(car_copy,columns=['Make'],prefix=['Make'])
car_copy = pd.get_dummies(car_copy,columns=['AccidentArea'],prefix=['AccidentArea'])
car_copy = pd.get_dummies(car_copy,columns=['Sex'],prefix=['Sex'])
car_copy = pd.get_dummies(car_copy,columns=['MaritalStatus'],prefix=['MartalStatus'])
car_copy = pd.get_dummies(car_copy,columns=['Fault'],prefix=['Fault'])
car_copy = pd.get_dummies(car_copy,columns=['PolicyType'],prefix=['PolicyType'])
car_copy = pd.get_dummies(car_copy,columns=['VehiclePrice'],prefix=['VehiclePrice'])
car_copy = pd.get_dummies(car_copy,columns=['Days:Policy-Accident'],prefix=['Days:Policy-Accident'])
car_copy = pd.get_dummies(car_copy,columns=['Days:Policy-Claim'],prefix=['Days:Policy-Claim'])
car_copy = pd.get_dummies(car_copy,columns=['PastNumberOfClaims'],prefix=['PastNumberOfClaims'])
car_copy = pd.get_dummies(car_copy,columns=['AgeOfVehicle'],prefix=['AgeOfVehicle'])
car_copy = pd.get_dummies(car_copy,columns=['AgeOfPolicyHolder'],prefix=['AgeOfPolicyHolder'])
car_copy = pd.get_dummies(car_copy,columns=['PoliceReportFiled'],prefix=['PoliceReportFiled'])
car_copy = pd.get_dummies(car_copy,columns=['WitnessPresent'],prefix=['WitnessPresent'])
car_copy = pd.get_dummies(car_copy,columns=['AgentType'],prefix=['AgentType'])
car_copy = pd.get_dummies(car_copy,columns=['NumberOfSuppliments'],prefix=['NumberOfSuppliments'])
car_copy = pd.get_dummies(car_copy,columns=['AddressChange-Claim'],prefix=['AddressChange-Claim'])
car_copy = pd.get_dummies(car_copy,columns=['NumberOfCars'],prefix=['NumberOfCars'])

car_copy.head()
print(type(car_copy))

<class 'pandas.core.frame.DataFrame'>


In [5]:
car_copy['RepNumber']= carDate['RepNumber']
car_copy['Deductible']= carDate['Deductible']
car_copy['DriverRating']= carDate['DriverRating']
car_copy['DayDiff']= carDate['DayDiff']

car_copy.head(3)


,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,AddressChange-Claim_under 6 months,NumberOfCars_1 vehicle,NumberOfCars_2 vehicles,NumberOfCars_3 to 4,NumberOfCars_5 to 8,NumberOfCars_more than 8,RepNumber,Deductible,DriverRating,DayDiff
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,12,300,1,0
1,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,15,400,4,5
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,7,400,3,13


In [6]:
#normalization of feature to bring the value in the range [0,1]
minMaxScale = MinMaxScaler()
carDateNormalized = minMaxScale.fit_transform(car_copy)
#print(car_df_Scale)

#converting numpyarry to dataframe
carDateNormalized = pd.DataFrame(carDateNormalized)
carDateNormalized.head(3)
print(type(carDateNormalized))

<class 'pandas.core.frame.DataFrame'>


/usr/local/anaconda35/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [7]:
#save to the normalized
#carDateNormalized.to_csv('finalNormalizedOneHot.csv',sep=' ',encoding='utf-8')

In [8]:
carArray= carDateNormalized.values
labels= carLable.values
print(type(carArray))

<class 'numpy.ndarray'>


In [9]:
#build the own data loader


In [10]:
#create the class Dataset which returns the data and labels
from torch.utils.data.dataset import Dataset, TensorDataset
class myDataset(Dataset):
    
    def __init__(self,features,lables,transform= None):
        dataTensor= []
        lableTensor= []
        dataSize= features.shape[0]
        
        for data in range(dataSize):
            feature= features[data,:]
            #feature= torch.from_numpy(feature).float()
            feature= torch.Tensor(feature)
            dataTensor.append(feature)
            
            lable= np.asanyarray(lables[data])
            lable= torch.from_numpy(lable).float()
           
            #lable= torch.Tensor(lable)
            #print("lable:",lable)
            #assert(False)
           
            lableTensor.append(lable)
        
        #put everything in features and lables
        self.features= dataTensor
        self.lables= lableTensor
    
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        
        feature= self.features[index]
        lable= self.lables[index]
        #print("get_item feature:",feature)
        #print("get_item lable:",lable)
        return feature,lable

In [11]:
#split the dataset into two part train and test
X_train,X_test,y_train,y_test = train_test_split(carArray,labels,random_state=3,test_size=0.25)
print('xtrain:',type(X_train))

xtrain: <class 'numpy.ndarray'>


In [12]:
#make the dataset for train and test loader
mytrainDataset= myDataset(X_train,y_train)
mytestDataset= myDataset(X_test,y_test)



In [13]:
feature,lable= mytrainDataset.__getitem__(0)
print("feature:",feature)
print("lable:",lable)
print(feature.size())

feature: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.2667, 0.2500, 1.0000, 0.0054])
lable: tensor(0.)
torch.Size([97])


In [14]:

trainLoader= torch.utils.data.DataLoader(mytrainDataset,batch_size=4,shuffle=True,num_workers=0)
testLoader= torch.utils.data.DataLoader(mytestDataset,batch_size=4,shuffle=True,num_workers=0)


In [15]:
#network architecture
class ANN(torch.nn.Module):
    def __init__(self):
        super(ANN,self).__init__()
        self.fc1= nn.Linear(97,6)
        self.fc2= nn.Linear(6,7)
        self.fc3= nn.Linear(7,9)
        self.fc4= nn.Linear(9,4)
        self.fc5= nn.Linear(4,5)
        self.fc6= nn.Linear(5,4)
        self.fc7= nn.Linear(4,4)
        self.fc8= nn.Linear(4,2)
        
    
    def forward(self,x):
        x= self.fc1(x)
        x= F.relu(x)
        x= self.fc2(x)
        x= F.relu(x)
        x= self.fc3(x)
        x= F.relu(x)
        x= self.fc4(x)
        x= F.relu(x)
        x= self.fc5(x)
        x= F.relu(x)
        x= self.fc6(x)
        x= F.relu(x)
        x= self.fc7(x)
        x= F.relu(x)
        x= self.fc8(x)
        x= F.sigmoid(x)
        return x
        

In [16]:
#code on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model= ANN()
model= model.to(device)

cuda:0


In [17]:
#define a loss function
criterion= nn.CrossEntropyLoss()
optimizer= optim.SGD(model.parameters(),lr=0.01,momentum = 0.9)
criterion= criterion.to(device) 

In [25]:
def testModle(model,epochNo):
    correct= 0
    total= 0
    lableList= []
    predictedList= []
    #lableList= lableList.to(device)
    #predictedList= predictedList.to(device)
    for i,(feature,lable) in enumerate(testLoader,0):
            #gets the inputs
            inputs= feature
            lables= lable
            lables= lables.type(torch.LongTensor)
            
            inputs = inputs.to(device)
            lables = lables.to(device)

            output= model(inputs)
            _,prediction= torch.max(output.data,1)
            correct += (prediction == lables).sum()

            total+= lables.size(0)
            lableList.append(lables)
            predictedList.append(prediction)

       
    
    trueLable= []
    predictedLable= []
    listLength= len(lableList)
    
    for i in range(listLength):
        size= len(lableList[i])
        for j in range(size):
            trueLable.append(lableList[i][j].cpu().numpy())
            predictedLable.append(predictedList[i][j].cpu().numpy())
    totalLength= len(trueLable)
    TP=FP=FN=TN= 0
    for i in range(totalLength):
        if(trueLable[i]==1 and predictedLable[i]==1):
            TP += 1
        elif(trueLable[i]==1 and predictedLable[i]==0):
            FN += 1
        elif(trueLable[i]==0 and predictedLable[i]==0):
            TN += 1
        elif(trueLable[i]==0 and predictedLable[i]==1):
            FP += 1
    print("\nResult for epoch: ",epochNo)
    print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
    print("Sensitivity:",TP/(TP+FN)*100)
    print("Specificity:",TN/(TN+FP)*100)
    print("\n")

    print("\nTP:",TP)
    print("\nFN:",FN)
    print("\nFP:",FP)
    print("\nTN:",TN)


In [19]:
#tarin the network
for epoch in range(10):
    running_loss = 0.0
    for i,(feature,lable) in enumerate(trainLoader,0):
        #gets the inputs
                
        inputs= torch.tensor(feature, requires_grad=True)
        lables= torch.tensor(lable, requires_grad=True)
        lables= lables.type(torch.LongTensor)
        
        inputs = inputs.to(device)
        lables = lables.to(device)
        
        #zero the parameter gradients
        optimizer.zero_grad()
        
        #forward+backward+optimize
        output = model(inputs)
        loss = criterion(output,lables)
        loss.backward()
        optimizer.step()
        
        #print the statistics
        running_loss += loss.item()
        if i%1000 == 0:              #print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0
    testModle(model,epoch+1)
        
print("finished Training")


/usr/local/anaconda35/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,     1] loss: 0.001
[1,  1001] loss: 0.405
[1,  2001] loss: 0.375

Result for epoch:  1
Accuracy: 93.61867704280155
Sensitivity: 0.0
Specificity: 100.0


[2,     1] loss: 0.001
[2,  1001] loss: 0.368
[2,  2001] loss: 0.372

Result for epoch:  2
Accuracy: 93.61867704280155
Sensitivity: 0.0
Specificity: 100.0


[3,     1] loss: 0.000
[3,  1001] loss: 0.372
[3,  2001] loss: 0.372

Result for epoch:  3
Accuracy: 93.61867704280155
Sensitivity: 0.0
Specificity: 100.0


[4,     1] loss: 0.000
[4,  1001] loss: 0.375
[4,  2001] loss: 0.368

Result for epoch:  4
Accuracy: 93.61867704280155
Sensitivity: 0.0
Specificity: 100.0


[5,     1] loss: 0.001
[5,  1001] loss: 0.373
[5,  2001] loss: 0.372

Result for epoch:  5
Accuracy: 93.61867704280155
Sensitivity: 0.0
Specificity: 100.0


[6,     1] loss: 0.000
[6,  1001] loss: 0.372
[6,  2001] loss: 0.371

Result for epoch:  6
Accuracy: 93.61867704280155
Sensitivity: 0.0
Specificity: 100.0


[7,     1] loss: 0.000
[7,  1001] loss: 0.371
[7,  2001] l

In [26]:
#train on the training set
testModle(model,1);

/usr/local/anaconda35/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")



Result for epoch:  1
Accuracy: 93.61867704280155
Sensitivity: 0.0
Specificity: 100.0



TP: 0

FN: 246

FP: 0

TN: 3609
